In [2]:
import optuna as opt
import pickle as pkl

In [3]:
with open('results/GB1_hyperopt_results.pkl', 'rb') as file: 
    gb1_opt_results = pkl.load(file)

/tmp/ipykernel_11219/155283431.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  gb1_opt_results = pkl.load(file)


In [8]:
gb1_opt_results['mlp'][0].best_value

0.013078873170615356

In [13]:
!pip install "numpy<2.0"


Defaulting to user installation because normal site-packages is not writeable


In [15]:
import torch